Import the surface temperature data, surface_temp.npy, for a small sphere from
 LMS. The temperature data is given for 1000 timesteps. Predict the temperatures for the
 next 10 timesteps using a recurrent neural network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from google.colab import drive

drive.mount("/content/drive")
file_path = ('/content/drive/MyDrive/Colab Notebooks/surface_temp.npy')

data = pd.DataFrame(np.load(file_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data.head()

,0
0,370.349970
1,370.652600
2,372.777706
3,374.208787
4,373.855544


In [92]:
import keras

x = np.array([np.linspace(0,999,num=1000)]).reshape(1000,)
y = np.array([data.iloc[:,-1]]).reshape(1000,)
print(x.shape)
print(y.shape)

def convert_to_timeseries(data, time):
  #reshape into [samples, timestep, features]
  timedata = data
  return timedata[...,np.newaxis, np.newaxis].astype(np.float32)

input = convert_to_timeseries(y, x)
print(input.shape)

(1000,)
(1000,)
(1000, 1, 1)


In [93]:
from sklearn.model_selection import train_test_split

n_steps = 998
X_train, y_train = input[:400, :n_steps], input[:400, -1]
X_valid, y_valid = input[400:600, :n_steps], input[400:600, -1]
X_test, y_test = input[600:, :n_steps], input[600:, -1]
y_test.shape

(400, 1)

In [104]:
from tensorflow import keras

model = keras.models.Sequential([
      keras.layers.SimpleRNN(1, activation='tanh', return_sequences=False, input_shape=[None, 1])
])

model.compile(loss="mse", optimizer="adam")

history = model.fit(X_train, y_train, epochs=100)

mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 140117.0938
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140075.6250 
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140113.8438 
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140122.8281 
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140089.7812 
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140123.0156 
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140051.6250 
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140112.2031 
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140122.0312  
Epoch 10/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 140096.3594  
Epoch 11/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140120.7969 
Epoch 12/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140142.5625 
Epoch 13/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140118.5938 
Epoch 14/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss:

In [102]:
input = convert_to_timeseries
X_new = np.array([np.linspace(0,9,num=10)]).reshape(10,1)
for step_ahead in range(10):
        y_pred_one = model.predict(X_new[:, step_ahead:])[:, np.newaxis, :]
        X = np.concatenate([X_new, y_pred_one], axis=1)
Y_pred = X[:, n_steps:]

print(Y_pred)


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None